In [1]:
#import sys
import os
sep = os.sep
# Добавить в путь до родительской папки\n",
#sys.path.append(os.path.join(sys.path[0], f'..{sep}'))
#sys.path.append(os.path.join(os.getcwd(), f'..{sep}'))

In [2]:
import pickle
from collections import Counter
#
import numpy as np
import matplotlib.pyplot as plt
#
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
#

%matplotlib inline

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 1 Introduction to recurrent neural networks (RNNs)

In [4]:
pass

# 2 Fundamentals of Natural Language Processing (NLP)

In [5]:
# Токенизация символов
text = 'It is unbelievably good!'

tokens = list(text)
print(tokens)

['I', 't', ' ', 'i', 's', ' ', 'u', 'n', 'b', 'e', 'l', 'i', 'e', 'v', 'a', 'b', 'l', 'y', ' ', 'g', 'o', 'o', 'd', '!']


In [6]:
text = 'Hi, there!'

tokens = list(text)
print(tokens)

['H', 'i', ',', ' ', 't', 'h', 'e', 'r', 'e', '!']


In [7]:
# Токенизация слов
text = 'It is unbelievably good!'
text = text.replace('!', ' !')

tokens = text.split(' ')
print(tokens)

['It', 'is', 'unbelievably', 'good', '!']


In [8]:
text = 'Hi, there!'
for x in list(',!'):
    text = text.replace(f'{x}', f' {x}')

tokens = text.split(' ')
print(tokens)

['Hi', ',', 'there', '!']


# 3 Prepare data to train the LSTM model

Загрузить текстовый файл Анны Карениной по ссылке https://github.com/LeanManager/NLP-PyTorch/tree/master/data.
Сохраните его как *anna.txt*. Открыть файл и удалить все после строки 39888, "END OF THIS PROJECT GUTENBERG EBOOK ANNA KARENINA".

## 3.1 Download the clean up the text

In [9]:
# Роман (на английском) "Анна Каренина"
with open(f'..{sep}data{sep}ch08{sep}anna.txt', 'r') as f:
    text = f.read()
words = text.split(' ') 
print( words[0:20] )

['Chapter', '1\n\n\nHappy', 'families', 'are', 'all', 'alike;', 'every', 'unhappy', 'family', 'is', 'unhappy', 'in', 'its', 'own\nway.\n\nEverything', 'was', 'in', 'confusion', 'in', 'the', "Oblonskys'"]


In [10]:
type(text), len(text)

(str, 1966150)

In [11]:
print(set(text.lower()))

{'_', '8', 'y', 'k', ')', 'u', '9', '7', 'c', ':', 'm', '3', 'g', 'r', 'f', 's', 'i', '?', 'z', '6', "'", '!', 'p', ' ', 't', 'b', '"', 'o', 'j', 'q', '5', '4', '1', 'x', '0', 'e', '.', 'h', 'v', 'd', '\n', ',', 'w', '`', 'l', '2', 'a', 'n', '-', ';', '('}


In [12]:
# Очистить текст
clean_text = text.lower().replace('\n', ' ')
clean_text = clean_text.replace('-', ' ')

# Перед и после символов <,.:;?!$()/_&%*@'`> вставить пробелы
for x in ",.:;?!$()/_&%*@'`":
    clean_text = clean_text.replace(f'{x}', f' {x} ')

clean_text = clean_text.replace('"', ' " ') 

# Переводим в спсисок слов (токенов)
text = clean_text.split()

In [13]:
type(text), len(text)

(list, 437098)

In [14]:
word_counts = Counter(text)
#word_counts

In [15]:
# Получить уникальные слова (токены), отсортированные
# по частоте встречаемости в тексте
words = sorted(word_counts, key=word_counts.get, reverse=True) 
print(words[:10])

[',', '.', 'the', '"', 'and', 'to', 'of', 'he', "'", 'a']


In [16]:
text_length = len(text)
num_unique_words = len(words)
print(f'Text contains {text_length} words')
print(f'There are {num_unique_words} unique tokens')  

# Слово (токен) является ключом словаря
word_to_int = {v: k for k, v in enumerate(words)} 
# Число является ключом словаря (токенов)
int_to_word = {k: v for k, v in enumerate(words)}

Text contains 437098 words
There are 12778 unique tokens


In [17]:
print({k: v for k, v in word_to_int.items() if k in words[:10]})

{',': 0, '.': 1, 'the': 2, '"': 3, 'and': 4, 'to': 5, 'of': 6, 'he': 7, "'": 8, 'a': 9}


In [18]:
print({k: v for k, v in int_to_word.items() if v in words[:10]})

{0: ',', 1: '.', 2: 'the', 3: '"', 4: 'and', 5: 'to', 6: 'of', 7: 'he', 8: "'", 9: 'a'}


In [19]:
# Пример, как узнать номер слова (токена) по словарю
print(text[0:20])
print([word_to_int[w] for w in text[0:20]])  

['chapter', '1', 'happy', 'families', 'are', 'all', 'alike', ';', 'every', 'unhappy', 'family', 'is', 'unhappy', 'in', 'its', 'own', 'way', '.', 'everything', 'was']
[208, 2755, 280, 2981, 83, 31, 2419, 35, 202, 685, 362, 38, 685, 10, 236, 147, 166, 1, 149, 12]


In [20]:
print(word_to_int['anna'])

62


In [21]:
# Перевести весь текст в список индексов
word_ids = [word_to_int[w] for w in text]  
#print(word_ids[0:20])
len(word_ids)

437098

## 3.2 Create batches of training data

In [22]:
# Длина последовательности (100 выбрали произвольно)
seq_len = 100
xys = []

# Создать пары последовательностей (x, y) для обучения.
# Каждый x это последовательность со 100 индексами.
# Затем сдвигаем окно вправо на один токен
# и используем его в качестве целевого y.
for n in range(0, len(word_ids)-seq_len-1):
    x = word_ids[n : n+seq_len]
    y = word_ids[n+1 : n+seq_len+1]
    xys.append((torch.tensor(x), (torch.tensor(y))))

# Сдвигая последовательность на один токен вправо и используя его в качестве выходных данных,
# модель обучается предсказывать следующий токен, учитывая предыдущие токены.

In [23]:
len(xys)

436997

In [24]:
batch_size = 32

In [25]:
torch.manual_seed(42)
# Пакеты данных для обучения, по 32 пары (x, y) в каждом пакете.
loader = DataLoader(xys, batch_size=batch_size, shuffle=True)

x, y = next(iter(loader))
print(x); print(y);
print(x.shape,y.shape)

tensor([[  39,   31,    2,  ...,  688,  142,    7],
        [ 156, 5293,    0,  ...,   38,  330,    0],
        [   3,   97,    0,  ...,    0, 1774,   34],
        ...,
        [  16,  156,    9,  ...,  113,    5,  533],
        [   3,    4,   31,  ...,   98,    5,   98],
        [ 289,   19,   23,  ...,    9,  828,  550]])
tensor([[  31,    2, 2727,  ...,  142,    7,    0],
        [5293,    0,   16,  ...,  330,    0,    3],
        [  97,    0,    4,  ..., 1774,   34,    3],
        ...,
        [ 156,    9,  489,  ...,    5,  533,   27],
        [   4,   31,   25,  ...,    5,   98,    1],
        [  19,   23,    1,  ...,  828,  550,    1]])
torch.Size([32, 100]) torch.Size([32, 100])


# 4 Building and Train the LSTM Model

In [26]:
pass

## 4.1 Build an LSTM model

**Класс torch.nn.Embedding()** используется для создания embedding-слоя в нейронной сети. Embedding-слой это обучаемая таблица поиска, которая сопоставляет целочисленные индексы с плотными векторными представлениями (embedding).

При создании экземпляра torch.nn.Embedding() необходимо указать два основных параметра: num_embeddings, размер словаря (общее количество уникальных токенов), и embedding_dim, размер каждого embedding-вектора.

Внутри класс создает матрицу (или таблицу поиска) формы (num_embeddings, embedding_dim), где каждая строка соответствует embedding-вектору для конкретного индекса. Изначально эти embeddings инициализируются случайным образом, но изучаются и обновляются во время обучения с помощью backpropagation.

Когда передается тензор индексов в embedding-слой (во время прямого прохода сети), он ищет соответствующие embedding-векторы в таблице поиска и возвращает их.

In [27]:
# Класс имеет три слоя:
# слой embedding, слой LSTM и последний линейный слой.
class WordLSTM(nn.Module):
    def __init__(self,
                 # input_size в слое nn.LSTM
                 input_size=128,
                 # Длина embedding (вектора) для кодирование одного токена
                 n_embed=128,
                 # num_layers в слое nn.LSTM
                 n_layers=3,
                 # dropout в слое nn.LSTM
                 drop_prob=0.2):
        super().__init__()
        self.input_size = input_size
        self.n_embed = n_embed
        self.drop_prob = drop_prob
        self.n_layers = n_layers

        # Кол-во уникальных токенов
        vocab_size = len(word_to_int)
        # Тренировочные данные сначала проходят через embedding-слой
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=n_embed)
        
        self.lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.n_embed,
            num_layers=self.n_layers,
            dropout=self.drop_prob,
            batch_first=True
        )

        # На выходе получаем logits - по одному числу для каждого уникального токена
        self.fc = nn.Linear(input_size, vocab_size)
    
    def forward(self, x, hc):
        embed = self.embedding(x)
        # На каждом шаге входными данными являются текущий токен и предыдущее скрытое состояние,
        # а выходными данными - следующий токен и следующее скрытое состояние
        x, hc = self.lstm(embed, hc)
        x = self.fc(x)
        return x, hc        

    # Инициализация весов нулями для первого токена
    # во входной последовательности
    # n_seqs: batch_size
    def init_hidden(self, n_seqs):
        weight = next(self.parameters()).data
        
        return (
            weight.new(self.n_layers, n_seqs, self.n_embed).zero_(),
            weight.new(self.n_layers, n_seqs, self.n_embed).zero_()
        ) 

In [28]:
model = WordLSTM().to(device)
model

WordLSTM(
  (embedding): Embedding(12778, 128)
  (lstm): LSTM(128, 128, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=12778, bias=True)
)

In [29]:
lr = 0.0001

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# Не забываем, что внутриCrossEntropyLoss() уже зашита функция Softmax
loss_func = nn.CrossEntropyLoss()

## 4.2 Train the LSTM model

In [30]:
# Обучение занимает около 3-4 суток
'''
model.train()
n_epoch = 50

for epoch in range(1, n_epoch+1):
    t_loss = 0
    sh, sc = model.init_hidden(batch_size)
    for i, (x, y) in enumerate(loader):    
        if x.shape[0] == batch_size:
            inputs, targets = x.to(device), y.to(device)
            optimizer.zero_grad()
            output, (sh,sc) = model(inputs, (sh, sc))
            loss = loss_func(output.transpose(1, 2), targets)
            sh, sc= sh.detach(), sc.detach()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            t_loss += loss.item()
        if (i+1)%1000 == 0:
            print_loss = t_loss / (i+1)
            print(f'Epoch {epoch}, iteration {i+1}, average loss = {print_loss:.6f}')
'''
pass

Epoch 1, iteration 1000, average loss = 6.391811
Epoch 1, iteration 2000, average loss = 6.216393
Epoch 1, iteration 3000, average loss = 6.157208
Epoch 1, iteration 4000, average loss = 6.099291
Epoch 1, iteration 5000, average loss = 6.003661
Epoch 1, iteration 6000, average loss = 5.897637
Epoch 1, iteration 7000, average loss = 5.796952
Epoch 1, iteration 8000, average loss = 5.705112
Epoch 1, iteration 9000, average loss = 5.621706
Epoch 1, iteration 10000, average loss = 5.546037
Epoch 1, iteration 11000, average loss = 5.476927
Epoch 1, iteration 12000, average loss = 5.414001
Epoch 1, iteration 13000, average loss = 5.355855
Epoch 2, iteration 1000, average loss = 4.575553
Epoch 2, iteration 2000, average loss = 4.551744
Epoch 2, iteration 3000, average loss = 4.530076
Epoch 2, iteration 4000, average loss = 4.508861
Epoch 2, iteration 5000, average loss = 4.489376
Epoch 2, iteration 6000, average loss = 4.471129
Epoch 2, iteration 7000, average loss = 4.453864
Epoch 2, iterati

In [31]:
# Save Trained Model
dir_name = 'files'
model_file_name = 'wordLSTM.pth'
dict_file_name = 'word_to_int.p'
os.makedirs(dir_name, exist_ok=True)

torch.save(model.state_dict(), f'{dir_name}{sep}{model_file_name}')
with open(f'{dir_name}{sep}{dict_file_name}', 'wb') as fb:    
    pickle.dump(word_to_int, fb) 

# 5 Generate text with the trained LSTM model

In [32]:
pass

## 5.1 Generate text by predicting the next token

In [33]:
model.load_state_dict(torch.load(f'{dir_name}{sep}{model_file_name}', weights_only=True))

with open(f'{dir_name}{sep}{dict_file_name}', 'rb') as fb:    
    word_to_int = pickle.load(fb)

int_to_word = {v: k for k, v in word_to_int.items()}

In [34]:
def sample(model, prompt, length=200):
    model.eval()
    text = prompt.lower().split(' ')
    hc = model.init_hidden(1)
    # Количество токенов, которые необходимо сгенерировать
    length = length - len(text)

    for i in range(0, length):
        # При генерации следующего токена учитываем длину текущей последовательности.
        # Если она меньше 100 токенов, мы вводим всю последовательность в модель;
        # если она больше 100 токенов, в качестве входных данных используются
        # только последние 100 токенов последовательности.
        if len(text) <= seq_len:
            x = torch.tensor([[word_to_int[w] for w in text]])
        else:
            # Входные данные — текущая последовательность;
            # обрезаем ее, если она длиннее 100 токенов.
            x = torch.tensor([[word_to_int[w] for w in text[-seq_len:]]])            
    
        inputs = x.to(device)
        output, hc = model(inputs, hc)
        logits = output[0][-1]

        p = nn.functional.softmax(logits, dim=0).detach().cpu().numpy()
        # Первый параметр метода указывает диапазон выбора, который в данном случае равен len(logits) = 12778.
        # Это означает, что модель будет случайным образом выбирать целое число от 0 до 12777,
        # причем каждое целое число будет соответствовать разному токену в словаре.
        # Второй параметр, p, представляет собой массив, содержащий 12778 элементов,
        # где каждый элемент обозначает вероятность выбора соответствующего токена из словаря.
        # Токены с более высокой вероятностью в этом массиве будут выбраны с большей вероятностью.
        idx = np.random.choice(len(logits), p=p)
        text.append(int_to_word[idx])

    text = ' '.join(text)
    for m in ",.:;?!$()/_&%*@'`":
        text = text.replace(f' {m}', f'{m} ')
    
    text = text.replace('"  ', '"')   
    text = text.replace("'  ", "'")  
    text=text.replace('" ', '"')   
    text=text.replace("' ", "'")     
    return text

In [35]:
torch.manual_seed(42)
np.random.seed(42)
gen_text = sample(model, prompt='Anna and the prince')
print(gen_text)

anna and the prince went across kitty to have to a lump.  anna had born waiting at the first day.  but that doctor was being brought on them,  and each one after another.  early paces away,  and then go so far to katavasov,  or it was only to cover home along every day again,  and went out to the pavilion to the elegant people where the muffled were come,  but come to ask her there to stand.  ""so i tell you if you have,  we can go on talking about the last time.  ""ah,  what are you going to have mercy over?  it's ten years given money,  "he said to veslovsky,  talking to his brother and speaking at once being very tiresome if he said a great deal.  "and i imagine that that i maintain her so far to me.  i am being himself alone,  and not cold with his mother.  he had certainly see everything the evening,  while i had told me i am but a wicked science creature,  and


## 5.2 Temperature and top-K sampling in text generation

In [36]:
# Более низкая temperature (ниже 1; например, 0,8) приводит к меньшему количеству вариаций,
# делая модель более детерминированной и консервативной, благоприятствуя более вероятным выборам.
# И наоборот, более высокая temperature (выше 1; например, 1,5) повышает вероятность выбора
# маловероятных слов при генерации текста.
#
# Выборка top_K — еще один метод влияния на выходные данные. Этот подход подразумевает выбор следующего слова
# из K самых вероятных вариантов, предсказанных моделью. Распределение вероятностей усекается, чтобы включить
# только K самых вероятных слов. При небольшом значении K, например 5, выбор модели ограничен несколькими
# высоковероятными словами, что приводит к более предсказуемым и связным,
# но потенциально менее разнообразным и интересным выходным данным.
def generate(model, prompt, top_k=None, length=200, temperature=1):
    model.eval()
    text = prompt.lower().split(' ')
    hc = model.init_hidden(1)
    length = length - len(text)    
    
    for i in range(0, length):
        if len(text) <= seq_len:
            x = torch.tensor([[word_to_int[w] for w in text]])
        else:
            x = torch.tensor([[word_to_int[w] for w in text[-seq_len:]]])    
        
        inputs = x.to(device)
        output, hc = model(inputs, hc)
        logits = output[0][-1]
        
        # scale the logits with the temperature 
        logits = logits/temperature
        p = nn.functional.softmax(logits, dim=0).detach().cpu()    
        
        if top_k is None:
            idx = np.random.choice(len(logits), p=p.numpy())
        else:
            # Оставить только top_K наиболее вероятных кандидатов
            ps, tops = p.topk(top_k)
            ps = ps / ps.sum()
            # Выбирать следующий токен из top_K кандидатов
            idx = np.random.choice(tops, p=ps.numpy())          
        text.append(int_to_word[idx])
    
    text = ' '.join(text)
    for m in ",.:;?!$()/_&%*@'`":
        text = text.replace(f' {m}', f'{m} ')
    
    text = text.replace('"  ', '"')   
    text = text.replace("'  ", "'")  
    text = text.replace('" ', '"')   
    text = text.replace("' ", "'")     
    return text

In [37]:
# Следующий токен с использованием настроек по умолчанию
prompt = "I ' m not going to see"
torch.manual_seed(42)
np.random.seed(42)

for _ in range(10):
    # Добавить к подсказке только один дополнительный токен
    gen_text = generate(model, prompt, top_k=None, length=len(prompt.split(' '))+1, temperature=1)
    print(gen_text)

i'm not going to see you
i'm not going to see myself
i'm not going to see him
i'm not going to see you
i'm not going to see it
i'm not going to see it
i'm not going to see her
i'm not going to see your
i'm not going to see you
i'm not going to see him


In [38]:
# Следующий токен с использованием консервативных прогнозов
prompt = "I ' m not going to see"
torch.manual_seed(42)
np.random.seed(42)

for _ in range(10):
    # Добавить к подсказке только один дополнительный токен
    gen_text = generate(model, prompt, top_k=3, length=len(prompt.split(' '))+1, temperature=0.5)
    print(gen_text)

i'm not going to see you
i'm not going to see her
i'm not going to see you
i'm not going to see you
i'm not going to see you
i'm not going to see you
i'm not going to see you
i'm not going to see you
i'm not going to see you
i'm not going to see you


In [39]:
torch.manual_seed(42)
np.random.seed(42)

gen_text = generate(model, prompt='Anna and the prince', top_k=3, temperature=0.5)
print(gen_text)

anna and the prince went to the door.  she was silent,  and she began walking up to her head,  and with rapid steps she went to her husband.  "i am very glad to tell you that.  ""i'm not going to say,  "said kitty,  smiling,  and holding her up,  she went out to dress,  and began walking up and down.  "what do you know?  ""yes,  i'm not going to say good bye,  "said stepan arkadyevitch,  getting up.  "well,  what is it?  ""i've not seen you long ago,  "said stepan arkadyevitch,  smiling.  "well,  what do you say?  ""oh,  no,  but how can you say that?  "said levin.  "oh,  yes!  "said levin,  smiling.  "oh,  yes,  sir.  ""well,  what do you think?  ""oh,  i shall have known you.  ""yes,  but how is it you
